In [1]:
!pip install openpyxl

In [2]:
import boto3
import pandas as pd
import numpy as np

## Preprocess the dataset
def preprocess_data(file_path):  
  df = pd.read_excel(file_path)
  ## Convert to datetime columns
  df["firstorder"]=pd.to_datetime(df["firstorder"],errors='coerce')
  df["lastorder"] = pd.to_datetime(df["lastorder"],errors='coerce')
  ## Drop Rows with null values
  df = df.dropna()    
  ## Create Column which gives the days between the last order and the first order
  df["first_last_days_diff"] = (df['lastorder']-df['firstorder']).dt.days
  ## Create Column which gives the days between when the customer record was created and the first order
  df['created'] = pd.to_datetime(df['created'])
  df['created_first_days_diff']=(df['created']-df['firstorder']).dt.days
  ## Drop Columns
  df.drop(['custid','created','firstorder','lastorder'],axis=1,inplace=True)
  ## Apply one hot encoding on favday and city columns
  df = pd.get_dummies(df,prefix=['favday','city'],columns=['favday','city'])
  return df

## Set the required configurations
model_name = "churn_model"
env = "dev"
## S3 Bucket
default_bucket = "customer-churn-sm-pipeline"
## Preprocess the dataset
storedata = preprocess_data("storedata_total.xlsx")

/opt/conda/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
storedata.head()

,retained,esent,eopenrate,eclickrate,avgorder,ordfreq,paperless,refill,doorstep,first_last_days_diff,...,favday_Monday,favday_Saturday,favday_Sunday,favday_Thursday,favday_Tuesday,favday_Wednesday,city_BLR,city_BOM,city_DEL,city_MAA
0,0,29,100.000000,3.448276,14.52,0.000000,0,0,0,0,...,True,False,False,False,False,False,False,False,True,False
1,1,95,92.631579,10.526316,83.69,0.181641,1,1,1,1024,...,False,False,False,False,False,False,False,False,True,False
2,0,0,0.000000,0.000000,33.58,0.059908,0,0,0,217,...,False,False,False,False,False,True,False,False,True,False
3,0,0,0.000000,0.000000,54.96,0.000000,0,0,0,0,...,False,False,False,True,False,False,False,True,False,False
4,1,30,90.000000,13.333333,111.91,0.008850,0,0,0,791,...,True,False,False,False,False,False,False,True,False,False


In [4]:
storedata.columns

Index(['retained', 'esent', 'eopenrate', 'eclickrate', 'avgorder', 'ordfreq',
       'paperless', 'refill', 'doorstep', 'first_last_days_diff',
       'created_first_days_diff', 'favday_Friday', 'favday_Monday',
       'favday_Saturday', 'favday_Sunday', 'favday_Thursday', 'favday_Tuesday',
       'favday_Wednesday', 'city_BLR', 'city_BOM', 'city_DEL', 'city_MAA'],
      dtype='object')

In [5]:
storedata.drop("retained", axis=1)

,esent,eopenrate,eclickrate,avgorder,ordfreq,paperless,refill,doorstep,first_last_days_diff,created_first_days_diff,...,favday_Monday,favday_Saturday,favday_Sunday,favday_Thursday,favday_Tuesday,favday_Wednesday,city_BLR,city_BOM,city_DEL,city_MAA
0,29,100.000000,3.448276,14.52,0.000000,0,0,0,0,-317,...,True,False,False,False,False,False,False,False,True,False
1,95,92.631579,10.526316,83.69,0.181641,1,1,1,1024,-103,...,False,False,False,False,False,False,False,False,True,False
2,0,0.000000,0.000000,33.58,0.059908,0,0,0,217,-59,...,False,False,False,False,False,True,False,False,True,False
3,0,0.000000,0.000000,54.96,0.000000,0,0,0,0,-157,...,False,False,False,True,False,False,False,True,False,False
4,30,90.000000,13.333333,111.91,0.008850,0,0,0,791,-2,...,True,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30796,6,83.333333,16.666667,55.68,0.000000,1,0,1,0,0,...,False,False,False,False,False,False,False,False,False,True
30797,13,69.230769,7.692308,35.00,0.000000,1,0,0,0,0,...,False,True,False,False,False,False,False,False,False,True
30798,8,75.000000,37.500000,52.03,0.500000,1,1,0,4,0,...,False,True,False,False,False,False,False,False,False,True
30799,6,33.333333,0.000000,40.20,0.000000,1,0,0,0,0,...,False,False,False,False,False,False,False,True,False,False


In [6]:
def split_datasets(df):
    #y=df.pop("retained")
    #X_pre = df
    y = df['retained']
    X_pre = df.drop("retained", axis=1)
    y_pre = y.to_numpy().reshape(len(y),1)
    feature_names = list(X_pre.columns)
    X= np.concatenate((y_pre,X_pre),axis=1)
    np.random.shuffle(X)
    train,validation,test=np.split(X,[int(.7*len(X)),int(.85*len(X))])
    return feature_names,train,validation,test

# Split dataset
feature_names,train,validation,test = split_datasets(storedata)

# Save datasets in Amazon S3
default_bucket = "sagemaker-us-east-2-582465847031"
pd.DataFrame(train).to_csv(f"s3://{default_bucket}/data/train/train.csv",header=False,index=False)
pd.DataFrame(validation).to_csv(f"s3://{default_bucket}/data/validation/validation.csv",header=False,index=False)
pd.DataFrame(test).to_csv(f"s3://{default_bucket}/data/test/test.csv",header=False,index=False)


In [9]:
!pip install sagemaker

In [28]:
from sagemaker.inputs import TrainingInput
import sagemaker
from sagemaker.tuner import ContinuousParameter, IntegerParameter, CategoricalParameter, HyperparameterTuner

In [29]:
# Training and Validation Input for SageMaker Training job
s3_input_train = TrainingInput(
    s3_data=f"s3://{default_bucket}/data/train/",content_type="csv")
s3_input_validation = TrainingInput(
    s3_data=f"s3://{default_bucket}/data/validation/",content_type="csv")

# Hyperparameter used
fixed_hyperparameters = {
    "eval_metric":"auc",
    "objective":"binary:logistic",
    "num_round":"100",
    "rate_drop":"0.3",
    "tweedie_variance_power":"1.4"
}

# Use the built-in SageMaker algorithm

sess = sagemaker.Session()
container = sagemaker.image_uris.retrieve("xgboost","us-east-2","0.90-2")
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()

estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    hyperparameters=fixed_hyperparameters,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/output".format(default_bucket),
    sagemaker_session=sagemaker_session
)

hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}
objective_metric_name = "validation:auc"
tuner = HyperparameterTuner(
estimator, objective_metric_name,
hyperparameter_ranges,max_jobs=10,max_parallel_jobs=2)

# Tune
tuner.fit({
    "train":s3_input_train,
    "validation":s3_input_validation
    },include_cls_metadata=False)

## Explore the best model generated
tuning_job_result = boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
)

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" %job_count)
## 10 training jobs have completed

## Get the best training job

from pprint import pprint
if tuning_job_result.get("BestTrainingJob",None):
    print("Best Model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.............................................................................!
10 training jobs have completed
Best Model found so far:
{'CreationTime': datetime.datetime(2025, 11, 11, 23, 55, 21, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.9797090291976929},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2025, 11, 11, 23, 55, 58, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-2:582465847031:training-job/sagemaker-xgboost-251111-2349-010-19c2a86f',
 'TrainingJobName': 'sagemaker-xgboost-251111-2349-010-19c2a86f',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2025, 11, 11, 23, 55, 24, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '1.8111728655526376',
                          'eta': '0.15931534104301992',
                          'max_depth': '8',
                          'min_child_weight': 